In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
uploaded = files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d START-UMD/gtd


!unzip gtd.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/START-UMD/gtd
License(s): other
  0% 0.00/28.7M [00:00<?, ?B/s]
100% 28.7M/28.7M [00:00<00:00, 961MB/s]
Archive:  gtd.zip
  inflating: globalterrorismdb_0718dist.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import os

sns.set()

In [ ]:
df = pd.read_csv('globalterrorismdb_0718dist.csv', encoding='latin1')

<ipython-input-4-393bafa96148>:2: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('globalterrorismdb_0718dist.csv', encoding='latin1')


In [ ]:
df_model = df[['iyear', 'country_txt', 'region_txt', 'attacktype1_txt',
               'target1', 'weaptype1_txt', 'nkill', 'success']]

df_model = df_model.dropna(subset=['nkill'])

le = LabelEncoder()
for col in ['country_txt', 'region_txt', 'attacktype1_txt', 'target1', 'weaptype1_txt']:
    df_model[col] = le.fit_transform(df_model[col].astype(str))

X = df_model.drop('success', axis=1)
y = df_model['success']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df_model, x='attacktype1_txt', order=df_model['attacktype1_txt'].value_counts().index[:10])
plt.title('Top 10 Attack Types')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("attack_types.png")
plt.close()

plt.figure(figsize=(10,5))
sns.countplot(data=df_model, x='region_txt', order=df_model['region_txt'].value_counts().index)
plt.title('Number of Attacks per Region')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("region_attacks.png")
plt.close()

plt.figure(figsize=(8,5))
sns.histplot(df_model['nkill'], bins=50, kde=True)
plt.title('Distribution of Fatalities')
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig("fatalities_distribution.png")
plt.close()

plt.figure(figsize=(5,5))
labels = ['Failed', 'Successful']
sizes = df_model['success'].value_counts()
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Success Rate of Attacks')
plt.axis('equal')
plt.savefig("success_rate.png")
plt.close()

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'ANN': MLPClassifier(hidden_layer_sizes=(50,), max_iter=500)
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append({'Model': name, 'Accuracy': acc})

results_df = pd.DataFrame(results)
results_df.to_csv('model_accuracies.csv', index=False)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data=results_df, x='Model', y='Accuracy')
plt.title('Accuracy Comparison Between Models')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig("accuracy_comparison.png")
plt.close()

In [ ]:
os.makedirs('Data/Result', exist_ok=True)
y_test.to_csv('Data/Result/y_test.csv', index=False)

for name, model in models.items():
    preds = model.predict(X_test)
    pd.DataFrame(preds, columns=[f'{name}_Prediction']).to_csv(f'Data/Result/{name.replace(" ", "_")}_pred.csv', index=False)